In [ ]:
import pandas as pd
import time
import urllib.request
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
import random
from random import choice #This library helps pick a random item from a list
from selenium.webdriver import Chrome
import bs4 as bs
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import csv

In [ ]:
Answered_Questions=[]
Price=[]
Discount=[]
customer_review_count=[]
rank=[]

# reading the individual files 
df_raw=pd.read_csv("electronicsru.csv",encoding="latin1")

# iterating through all the top products in a category

for i in range(0,100):
    
    url='https://www.amazon.com/'+df_raw["product_link"][i]
    
    print(url)
    user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246']
    
    try:
        
        class MyOpener(FancyURLopener, object):
                version = choice(user_agents)

        myopener = MyOpener()
        page=myopener.open(url)

        time.sleep(random.uniform(20,25))
        html=page.read().decode('utf-8')
        soup=bs.BeautifulSoup(html)
        
    except:
        Answered_Questions.append("No Answered Questions")
        Price.append("No Price")
        Discount.append("No Discount")
        customer_review_count.append("No Customer Reviews")
        
    # retrieve the answered questions
    try:

        Answered_Questions.append(soup.find_all("span",class_="celwidget")[0].find("span",class_="a-size-base").getText().replace("+","").split()[0])
    except:
        Answered_Questions.append("0")
    
    
    # Retrieve the price of the product
    if soup.find_all("span",class_="a-size-medium a-color-price priceBlockBuyingPriceString")!= []:
        Price.append(soup.find_all("span",class_="a-size-medium a-color-price priceBlockBuyingPriceString")[0].getText()[1:])
    elif soup.find_all("span",class_="a-size-medium a-color-price priceBlockDealPriceString")!= []:
        Price.append(soup.find_all("span",class_="a-size-medium a-color-price priceBlockDealPriceString")[0].getText()[1:])
    else:
        Price.append("0")

    # retrieve customer review count
    try:
        customer_review_count.append(soup.find_all("span",class_="cr-widget-FocalReviews")[0].find("span",class_="a-size-base").getText().split()[0])
    except:
        customer_review_count.append("0")
    
    # retrieve discount
    try:
        Discount.append(soup.find_all("td",class_="a-span12 a-color-price a-size-base priceBlockSavingsString")[0].getText().replace("\n","").split()[1].split('(')[1].split('%')[0])
    except:
        Discount.append("0")
    rank.append(i+1)

# appending all the lists to a dataframe
df = pd.DataFrame(list(zip(rank, Answered_Questions, Price , customer_review_count, Discount)), 
            columns =['rank', 'Answered_Questions', 'Price' , 'customer_review_count', 'Discount'])

# exporting dataframe to csv file
df.to_csv('dataframe_category.csv', index = False)
